In [42]:
pID = 3
pID = 'sub-' + "%03d" % (pID)

In [43]:
import os

import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier as ABC

from sklearn.model_selection import cross_val_score, KFold

import pickle, json, os
import scipy.io
from bci_funcs import windowed_mean, base_correct, select_mean, slope

path = '/Volumes/Lukas_Gehrke/fastReach/data/eeglab2python'

# Preconscious Augmentation

This script trains the classifier for a brain-computer interface that controls electrical muscle stimulation in the preconscious augmentation experiment.
The functions used to build the feature vectors are the same that are used for the online application and are found in 'bci_funcs'

A two class linear discriminant model is fitted to idle and pre-movement EEG training data. The model and a channel selection is saved.

In [44]:
# load data
pre_move = scipy.io.loadmat(path+os.sep+pID+os.sep+'pre_move_Baseline.mat')
idle = scipy.io.loadmat(path+os.sep+pID+os.sep+'idle_Baseline.mat')
all_idle = idle['idle']
all_pre_move = pre_move['pre_move']

# Compute Features & Model fitting

In [45]:
nr_chans_arr = np.arange(2,21,2) # select one and then save the model

data_srate = 250
windows = 10
baseline = data_srate/windows

chan_names = pd.read_csv(path+os.sep+pID+os.sep+'sel_chans_names.csv')
all_chans = pd.read_csv(path+os.sep+pID+os.sep+'sel_chans.csv', header=None)

feat = 'slope' # slope or slope_2d

for nr_chans in nr_chans_arr:
    print(str(nr_chans) + ' channels')

    chans = np.squeeze(np.array(all_chans[0:nr_chans]) - 1).tolist()
    # print(chan_names.Var1[chans])

    idle = all_idle[chans, :, :]
    pre_move = all_pre_move[chans, :, :]

    if feat == 'slope':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0]))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0]))

    elif feat == 'slope2d':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0]*2))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0]*2))

    elif feat == 'windowed_means':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0] * (windows)))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0] * (windows)))

    for trial_ix in range(0, pre_move.shape[2]):
        if feat == 'slope':
            pre_move_feat[trial_ix, :] = slope(pre_move[:,:,trial_ix], 'linear').flatten()
        
        elif feat == 'slope2d':
            pre_move_feat[trial_ix, :idle.shape[0]] = slope(pre_move[:,:150,trial_ix], 'linear').flatten()
            pre_move_feat[trial_ix, idle.shape[0]:] = slope(pre_move[:,150:,trial_ix], 'linear').flatten()

        elif feat == 'windowed_means':
            tmp = base_correct(pre_move[:,:,trial_ix], baseline-1)
            pre_move_feat[trial_ix, :] = windowed_mean(tmp, windows).flatten()

    for trial_ix in range(0, idle.shape[2]):
        if feat == 'slope':
            idle_feat[trial_ix, :] = slope(idle[:,:,trial_ix], 'linear').flatten()    
        
        elif feat == 'slope2d':
            idle_feat[trial_ix, :idle.shape[0]] = slope(idle[:,:150,trial_ix], 'linear').flatten()
            idle_feat[trial_ix, idle.shape[0]:] = slope(idle[:,150:,trial_ix], 'linear').flatten()

        elif feat == 'windowed_means':
            tmp = base_correct(idle[:,:,trial_ix], baseline-1)
            idle_feat[trial_ix, :] = windowed_mean(tmp, windows).flatten()

    data = np.concatenate((pre_move_feat, idle_feat), axis = 0)
    pre_move_class = np.ones((pre_move_feat.shape[0], 1))
    idle_class = np.zeros((idle_feat.shape[0], 1))
    classes = np.concatenate((pre_move_class, idle_class)).ravel()

    clf = LDA(solver='eigen', shrinkage='auto')
    clf.fit(data, classes)
    kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    print("LDA accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    # clf = ABC()
    # clf.fit(data, classes)
    # kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    # cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    # print("ABC accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    # clf = QDA()
    # clf.fit(data, classes)
    # kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    # cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    # print("QDA accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    # clf = RFC()
    # clf.fit(data, classes)
    # kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    # cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    # print("RFC accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

2 channels
LDA accuracy: 0.7214285714285715 +/- 0.05714285714285711
4 channels
LDA accuracy: 0.7214285714285715 +/- 0.03499271061118824
6 channels
LDA accuracy: 0.7785714285714286 +/- 0.05714285714285712
8 channels
LDA accuracy: 0.7571428571428571 +/- 0.05248906591678237
10 channels
LDA accuracy: 0.7857142857142858 +/- 0.05050762722761051
12 channels
LDA accuracy: 0.75 +/- 0.04517539514526254
14 channels
LDA accuracy: 0.7785714285714286 +/- 0.07284313590846832
16 channels
LDA accuracy: 0.7928571428571429 +/- 0.06144518047887588
18 channels
LDA accuracy: 0.7928571428571428 +/- 0.08268454930564446
20 channels
LDA accuracy: 0.7785714285714286 +/- 0.07284313590846832


# Real-time Detection

LDA accuracy: 0.7785714285714286 +/- 0.07284313590846832


Threshold value is: 0.4926336042133755
F1 Score: 0.855072463768116


In [47]:
nr_chans = 18
target_class = 1.0
classifier_update_rate = 5 # samples

### COPY IN WHERE BEST CLASSIFICATION PERFORMANCE WAS ACHIEVED ###

clf = LDA(solver='eigen', shrinkage='auto')
clf.fit(data, classes)
kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, data, classes, cv=kfolds)
print("LDA accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

### END ###

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

predictions_prob = clf.predict_proba(data)
pred_classes = clf.predict(data)
false_positive_rate, recall, thresholds = roc_curve(classes, predictions_prob[:,1])
roc_auc = auc(false_positive_rate, recall)
plot_roc_curve(false_positive_rate, recall)

optimal_idx = np.argmax(recall - false_positive_rate)
optimal_threshold = thresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)

from sklearn.metrics import f1_score
print("F1 Score: "+ str(f1_score(classes, pred_classes)))

filename = path+os.sep+pID+os.sep+'model_'+pID+'_eeg.sav'
pickle.dump(clf, open(filename, 'wb'))

# threshold = cv_results.mean()
threshold = np.min(thresholds[false_positive_rate<.1])
print("Threshold used is:", threshold)

bci_params = dict(((k, eval(k)) for k in ('chans', 'windows', 'baseline', 'target_class', 'threshold', 'data_srate', 'classifier_update_rate')))
with open(path+os.sep+pID+os.sep+'bci_params.json', 'w') as f:
    json.dump(bci_params, f)